In [23]:
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
import torch
import pickle
import os
from PIL import Image
from tqdm.auto import tqdm

In [15]:
import numpy
numpy.__version__

'1.26.0'

In [16]:
# set model
device = "cuda" if torch.cuda.is_available() else "cpu"

model = resnet50(weights=ResNet50_Weights.DEFAULT).to(device).eval()

In [17]:
# remove fc layer

embedd_model = torch.nn.Sequential(*list(model.children())[:-1])

In [18]:
# set imagenet transforms

image_transforms = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

In [19]:
dataset_path = "../data/dataset"
features_lst = []

for fldr in tqdm(os.listdir(dataset_path)):
    fldr_path = os.path.join(dataset_path, fldr)
    for img in os.listdir(fldr_path):
        img_path = os.path.join(fldr_path, img)
        init_image = Image.open(img_path).convert("RGB")
        image = image_transforms(init_image).to(device)
        
        with torch.inference_mode():
            image_feat = embedd_model(image.unsqueeze(0)).squeeze(0).view(-1)
            image_feat = image_feat.cpu().numpy()
            
        image_data = {
            "path": img_path,
            "features": image_feat,
        }
        
        features_lst.append(image_data)
        
print("[INFO] Done!")

100%|██████████| 5/5 [04:37<00:00, 55.50s/it]

[INFO] Done!


In [24]:
# save features data to json file

feat_data_path = "../data/features.pkl"

with open(feat_data_path, "wb") as f:
    pickle.dump(features_lst, f)